In [1]:
### Enviroment Setup
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
import json
import asyncio

from typing import Dict, Any, List

# Assuming ADK imports are structured like this for the latest version
from google.adk.agents import LlmAgent
from google.adk.tools.function_tool import FunctionTool
from google.adk.models.google_llm import Gemini
from google.genai import types
from typing import Dict, Any, List

# Session & Memory Information

from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.memory import InMemoryMemoryService # Import MemoryService
from google.adk.runners import Runner
from google.adk.tools import load_memory # Tool to query memory
from google.genai.types import Content, Part


# Services must be shared across runners to share state and memory
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService() # Use in-memory for demo


# --- 1. Mock Tool Implementations ---
# In a real environment, these tools would interface with the MCP (Mission Control Platform)
# or custom external APIs (e.g., databases, microservices).

def mcp_order_management_tool(order_id: str) -> str:
    """
    Retrieves the current status and details for a given order ID from the MCP system.
    Returns a JSON string containing the order status, shipment date, and items.
    """
    # Mock data simulation
    if order_id == "ORD1001":
        return json.dumps({
            "order_id": order_id,
            "status": "In Transit",
            "shipment_date": "2024-11-25",
            "estimated_delivery": "2024-11-28",
            "items": ["Galaxy S24", "Wireless Charger"],
            "tracking_number": "TRK123456"
        })
    elif order_id == "ORD1002":
        return json.dumps({
            "order_id": order_id,
            "status": "Pending Resolution",
            "issue": "Payment validation failed",
            "action_required": "Customer needs to update payment information."
        })
    else:
        return json.dumps({"order_id": order_id, "status": "Not Found", "message": "Order ID is invalid or not recognized."})

# --- 2. Specialist Agents Definition (LlmAgent) ---

order_status_agent = LlmAgent(
    name='order_status_agent',
    model='gemini-2.5-flash-lite',
    description='Creates the specialist agent responsible for handling order status queries.',
    instruction='You are the Order Status Specialist. Your primary goal is to use the mcp_order_management_tool to fetch the requested order details and present the information clearly to the customer. Do not invent information. If an order is In Transit, provide the tracking number and estimated delivery.',
    tools=[mcp_order_management_tool],
    output_key='create_order_status_agent_findings',  # The result of this agent will be stored in the session state with this key.
)

print("✅ Created Order Status Agent.")

issue_resolution_agent=LlmAgent(
        name="issue_resolution_agent",
        instruction = (
        "You are the Issue Resolution Specialist. Your task is to analyze the customer's problem "
        "and provide policy-compliant solutions. If the order status is 'Pending Resolution', "
        "use the 'mcp_order_management_tool' to understand the root cause and advise the customer "
        "on the next steps required to resolve the issue."
        ),
        model='gemini-2.5-flash-lite',
        tools=[mcp_order_management_tool],
        output_key='issue_resolution_agent_findings', 
)

print("✅ Create issue resolution agent.")

memory_recall_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="MemoryRecallAgent",
    instruction="Answer the user's question. Use the 'load_memory' tool "
                "if the answer might be in past conversations.",
    tools=[load_memory] # Give the agent the tool
)

print("✅ Memory Recall Agent Created.")

# --- 3. Main Router Agent Orchestration (LlmAgent) ---
root_agent=LlmAgent(
    instruction = 'You are the Enterprise Order Management Router Agent. Your sole responsibility is to classify the users intent and route the query to the most appropriate specialist agent provided in your tools list. If the user asks for order status or tracking, route to order_status_agent. If the user expresses a complaint, wants to cancel, or needs help fixing an issue, route to issue_resolution_agent. For general, non-order related questions, answer directly and concisely.',
    name='router_agent',
    model="gemini-2.0-flash",
    sub_agents=[order_status_agent, issue_resolution_agent,memory_recall_agent],
    output_key="root_agent_findings",
)
print("✅ Router Agent Created..")

✅ Created Order Status Agent.
✅ Create issue resolution agent.
✅ Memory Recall Agent Created.
✅ Router Agent Created..


In [4]:
APP_NAME = "Enterprise App"
USER_ID = "Peter Jerald"

async def run_scenario():
    # --- Scenario ---
    
    print("--- Run 1 #: Starting runner ---")
    runner1 = Runner(
        agent=root_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service # Provide the memory service to the Runner
    )
    session1_id = "session_info#ID1"
    await runner1.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=session1_id)
    user_input1 = Content(parts=[Part(text="Please cancel my order ID : ORD1001 since it was pending for very long period.")], role="user")

    # Run the agent
    final_response_text = "(No final response)"
    async for event in runner1.run_async(user_id=USER_ID, session_id=session1_id, new_message=user_input1):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text = event.content.parts[0].text
    print(f"Agent 1 Response: {final_response_text}")

    # Get the completed session
    completed_session1 = await runner1.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=session1_id)

    # Add this session's content to the Memory Service
    print("\n--- Adding Session 1 to Memory ---")
    await memory_service.add_session_to_memory(completed_session1)
    print("Session added to memory.")

     # Run 2: Recall the information in a new session
    print("\n--- Run 2: Recalling Information ---")
    runner2 = Runner(
        # Use the second agent, which has the memory tool
        agent=root_agent,
        app_name=APP_NAME,
        session_service=session_service, # Reuse the same service
        memory_service=memory_service   # Reuse the same service
    )
    session2_id = "session_recall"
    await runner2.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=session2_id)
    user_input2 = Content(parts=[Part(text="Did I raise the request to cancel my order eariler? Order ID : ORD1001")], role="user")

    # Run the second agent
    final_response_text_2 = "(No final response)"
    async for event in runner2.run_async(user_id=USER_ID, session_id=session2_id, new_message=user_input2):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text_2 = event.content.parts[0].text
    print(f"Agent 2 Response: {final_response_text_2}")

In [5]:
await run_scenario()

--- Run 1 #: Starting runner ---


Agent 1 Response: I understand you wish to cancel your order ID ORD1001 due to a long pending period. However, I see that this order is currently 'In Transit' and is expected to be delivered by 2024-11-28. Unfortunately, orders that have already been shipped cannot be canceled.

You can choose to return the item once it's delivered if you no longer need it. Please let me know if you have any other questions.

--- Adding Session 1 to Memory ---
Session added to memory.

--- Run 2: Recalling Information ---


Agent 2 Response: The request to cancel your order ORD1001 was not raised. The order is currently in transit and is estimated to be delivered by 2024-11-28.
